In [48]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import os
import sys
import numpy as np
from ipywidgets import widgets
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import gmean
plt.style.use('seaborn-talk')


df_no_by = pd.read_pickle('../pickles/pair_dynamic_no_bypass.pkl')
for col in df_no_by.columns:
    print(col)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
1_bench
2_bench
pair_str
config_sim
gpusim_version
jobId
tot_runtime
stall_icnt_to_l2
stall_l2_to_icnt
stall_core_ldst
l1D_miss_rate
l2_miss_rate
l2_rshr_entry_fail
l2_rshr_merge_fail
l2_total_accesses
mem_count
empty_warp
stall_warp
idle_warp
scoreboard_warp
tot_warp_insn
sp_busy
dp_busy
int_busy
tensor_busy
sfu_busy
regs
smem
runtime
instructions
l2_bw
barrier_cycles
inst_empty_cycles
branch_cycles
scoreboard_cycles
stall_sp_cycles
stall_dp_cycles
stall_int_cycles
stall_tensor_cycles
stall_sfu_cycles
stall_mem_cycles
not_selected_cycles
cycles_per_issue
avg_mem_lat
avg_core_to_l2
avg_l2_to_core
avg_mrq_latency
dram_eff
dram_bw
row_buffer_locality
mem_idle
total_cmd
1_kidx
2_kidx
avg_dram_bw
1_intra
2_intra
1_conc_cta
2_conc_cta
sld
1_sld
2_sld
fairness
ws
index
pair_str_x
config_x
gpusim_version_x
jobId_x
tot_runtime_x
stall_icnt_to_l2_x
stall_l2_to_icnt_x
stall_core_ldst_x
l1D_miss_rate_x
l2_miss

In [49]:
df_no_by.drop(df_no_by.columns.difference(['1_bench','2_bench','pair_str','ws','config_sim']), 1, inplace=True)
df_no_by_ws_sorted = df_no_by.sort_values('ws', ascending=False).drop_duplicates(['pair_str'])
print(df_no_by_ws_sorted.to_string())

                1_bench              2_bench                                pair_str                                                                                                                              config_sim        ws
300      nvd_binomial-0           parb_lbm-0               nvd_binomial-0-parb_lbm-0   V100-PAE-CONCURRENT-SEP_RW-LSRR-PRINT_DEVICE_SYNC-CAP_3606625_CYCLE-INTRA_0:13:2_CTA-ENABLE_L2D_1:1:1-MIX_0:1:1_KIDX-NUM_0:1:1_KERNEL  2.106919
130          parb_lbm-0         rod_lavamd-0                 parb_lbm-0-rod_lavamd-0   V100-PAE-CONCURRENT-SEP_RW-LSRR-PRINT_DEVICE_SYNC-CAP_11767167_CYCLE-INTRA_0:2:6_CTA-ENABLE_L2D_1:1:1-MIX_0:1:1_KIDX-NUM_0:1:1_KERNEL  2.051409
374  nvd_blackscholes-0         parb_cutcp-0         nvd_blackscholes-0-parb_cutcp-0    V100-PAE-CONCURRENT-SEP_RW-LSRR-PRINT_DEVICE_SYNC-CAP_2007327_CYCLE-INTRA_0:8:2_CTA-ENABLE_L2D_1:1:1-MIX_0:1:1_KIDX-NUM_0:1:1_KERNEL  1.970186
45         parb_cutcp-0           parb_lbm-0                 parb_cutcp-0-pa

In [50]:
#df_no_dropped = df_no_by_ws_sorted.drop(df_no_by_ws_sorted.columns.difference(['ws']), 1, inplace=True)
#print(gmean(df_no_dropped))


In [51]:
df_by = pd.read_pickle('../pickles/pair_dynamic.pkl')


In [52]:
miss_rate_apps = ['nvd_blackscholes-0', 'rod_streamcluster-0', 'parb_lbm-0', 'nvd_sobol-0']
df_by = df_by.loc[(df_by['1_bench'].isin(miss_rate_apps)) | (df_by['2_bench'].isin(miss_rate_apps))]

df_by.drop(df_by.columns.difference(['1_bench','2_bench','pair_str','ws','config_sim']), 1, inplace=True)
df_by_ws_sorted = df_by.sort_values('ws', ascending=False).drop_duplicates(['pair_str'])

print(df_by_ws_sorted.to_string())


                 1_bench              2_bench                                pair_str        ws
547         parb_cutcp-0  rod_streamcluster-0        parb_cutcp-0-rod_streamcluster-0  1.977781
1110  nvd_blackscholes-0         rod_lavamd-0         nvd_blackscholes-0-rod_lavamd-0  1.955111
885   nvd_blackscholes-0         parb_cutcp-0         nvd_blackscholes-0-parb_cutcp-0  1.949153
922           parb_lbm-0         rod_lavamd-0                 parb_lbm-0-rod_lavamd-0  1.865304
824   nvd_blackscholes-0       nvd_interval-0       nvd_blackscholes-0-nvd_interval-0  1.803295
859       nvd_binomial-0           parb_lbm-0               nvd_binomial-0-parb_lbm-0  1.788531
14        nvd_binomial-0   nvd_blackscholes-0       nvd_binomial-0-nvd_blackscholes-0  1.704360
67        nvd_interval-0          nvd_sobol-0              nvd_interval-0-nvd_sobol-0  1.701685
904          nvd_sobol-0         parb_cutcp-0                nvd_sobol-0-parb_cutcp-0  1.696931
1127      nvd_binomial-0  rod_streamclus

In [53]:
#df_by_dropped = df_by_ws_sorted.drop(df_by_ws_sorted.columns.difference(['ws']), 1, inplace=True)
#print(gmean(df_by_dropped))

In [56]:
df_both = pd.merge(df_no_by_ws_sorted, df_by_ws_sorted, on=['1_bench', '2_bench'], suffixes=('_no_bypass', '_bypass'))
df_both

,1_bench,2_bench,pair_str_no_bypass,config_sim,ws_no_bypass,pair_str_bypass,ws_bypass
0,nvd_binomial-0,parb_lbm-0,nvd_binomial-0-parb_lbm-0,V100-PAE-CONCURRENT-SEP_RW-LSRR-PRINT_DEVICE_S...,2.106919,nvd_binomial-0-parb_lbm-0,1.788531
1,parb_lbm-0,rod_lavamd-0,parb_lbm-0-rod_lavamd-0,V100-PAE-CONCURRENT-SEP_RW-LSRR-PRINT_DEVICE_S...,2.051409,parb_lbm-0-rod_lavamd-0,1.865304
2,nvd_blackscholes-0,parb_cutcp-0,nvd_blackscholes-0-parb_cutcp-0,V100-PAE-CONCURRENT-SEP_RW-LSRR-PRINT_DEVICE_S...,1.970186,nvd_blackscholes-0-parb_cutcp-0,1.949153
3,parb_cutcp-0,parb_lbm-0,parb_cutcp-0-parb_lbm-0,V100-PAE-CONCURRENT-SEP_RW-LSRR-PRINT_DEVICE_S...,1.957132,parb_cutcp-0-parb_lbm-0,1.643440
4,parb_cutcp-0,rod_streamcluster-0,parb_cutcp-0-rod_streamcluster-0,V100-PAE-CONCURRENT-SEP_RW-LSRR-PRINT_DEVICE_S...,1.946291,parb_cutcp-0-rod_streamcluster-0,1.977781
...,...,...,...,...,...,...,...
61,rod_hotspot3d-0,rod_streamcluster-0,rod_hotspot3d-0-rod_streamcluster-0,V100-PAE-CONCURRENT-SEP_RW-LSRR-PRINT_DEVICE_S...,0.991526,rod_hotspot3d-0-rod_streamcluster-0,1.045723
62,parb_lbm-0,rod_hotspot3d-0,parb_lbm-0-rod_hotspot3d-0,V100-PAE-CONCURRENT-SEP_RW-LSRR-PRINT_DEVICE_S...,0.976057,parb_lbm-0-rod_hotspot3d-0,0.755676
63,nvd_blackscholes-0,parb_stencil-0,nvd_blackscholes-0-parb_stencil-0,V100-PAE-CONCURRENT-SEP_RW-LSRR-PRINT_DEVICE_S...,0.964579,nvd_blackscholes-0-parb_stencil-0,0.864314
64,parb_lbm-0,parb_stencil-0,parb_lbm-0-parb_stencil-0,V100-PAE-CONCURRENT-SEP_RW-LSRR-PRINT_DEVICE_S...,0.958417,parb_lbm-0-parb_stencil-0,0.669668


In [59]:
# spawn two more dataframes to compare results:
df_both['diff'] = df_both['ws_bypass'] - df_both['ws_no_bypass']
df_both['diff']

0    -0.318388
1    -0.186105
2    -0.021033
3    -0.313692
4     0.031489
        ...   
61    0.054197
62   -0.220381
63   -0.100265
64   -0.288749
65    0.018857
Name: diff, Length: 66, dtype: float64

In [60]:
# compare results
df_both['bypass_is_better'] = np.where(df_both['diff'] > 0, 'True', 'False')
df_both

,1_bench,2_bench,pair_str_no_bypass,config_sim,ws_no_bypass,pair_str_bypass,ws_bypass,diff,bypass_is_better
0,nvd_binomial-0,parb_lbm-0,nvd_binomial-0-parb_lbm-0,V100-PAE-CONCURRENT-SEP_RW-LSRR-PRINT_DEVICE_S...,2.106919,nvd_binomial-0-parb_lbm-0,1.788531,-0.318388,False
1,parb_lbm-0,rod_lavamd-0,parb_lbm-0-rod_lavamd-0,V100-PAE-CONCURRENT-SEP_RW-LSRR-PRINT_DEVICE_S...,2.051409,parb_lbm-0-rod_lavamd-0,1.865304,-0.186105,False
2,nvd_blackscholes-0,parb_cutcp-0,nvd_blackscholes-0-parb_cutcp-0,V100-PAE-CONCURRENT-SEP_RW-LSRR-PRINT_DEVICE_S...,1.970186,nvd_blackscholes-0-parb_cutcp-0,1.949153,-0.021033,False
3,parb_cutcp-0,parb_lbm-0,parb_cutcp-0-parb_lbm-0,V100-PAE-CONCURRENT-SEP_RW-LSRR-PRINT_DEVICE_S...,1.957132,parb_cutcp-0-parb_lbm-0,1.643440,-0.313692,False
4,parb_cutcp-0,rod_streamcluster-0,parb_cutcp-0-rod_streamcluster-0,V100-PAE-CONCURRENT-SEP_RW-LSRR-PRINT_DEVICE_S...,1.946291,parb_cutcp-0-rod_streamcluster-0,1.977781,0.031489,True
...,...,...,...,...,...,...,...,...,...
61,rod_hotspot3d-0,rod_streamcluster-0,rod_hotspot3d-0-rod_streamcluster-0,V100-PAE-CONCURRENT-SEP_RW-LSRR-PRINT_DEVICE_S...,0.991526,rod_hotspot3d-0-rod_streamcluster-0,1.045723,0.054197,True
62,parb_lbm-0,rod_hotspot3d-0,parb_lbm-0-rod_hotspot3d-0,V100-PAE-CONCURRENT-SEP_RW-LSRR-PRINT_DEVICE_S...,0.976057,parb_lbm-0-rod_hotspot3d-0,0.755676,-0.220381,False
63,nvd_blackscholes-0,parb_stencil-0,nvd_blackscholes-0-parb_stencil-0,V100-PAE-CONCURRENT-SEP_RW-LSRR-PRINT_DEVICE_S...,0.964579,nvd_blackscholes-0-parb_stencil-0,0.864314,-0.100265,False
64,parb_lbm-0,parb_stencil-0,parb_lbm-0-parb_stencil-0,V100-PAE-CONCURRENT-SEP_RW-LSRR-PRINT_DEVICE_S...,0.958417,parb_lbm-0-parb_stencil-0,0.669668,-0.288749,False


In [61]:
df_both['bypass_is_better'].value_counts()

False    49
True     17
Name: bypass_is_better, dtype: int64

In [66]:
print("average no bypass improvement is ", df_both[df_both['diff'] < 0]['diff'].mean())
print("max no bypass improvement is ", df_both[df_both['diff'] < 0]['diff'].min())
print("average bypass improvement is ", df_both[df_both['diff'] > 0]['diff'].mean())
print("max bypass improvement is ", df_both[df_both['diff'] > 0]['diff'].max())

 
average no bypass improvement is  -0.15709712216334737
max no bypass improvement is  -0.49358097024245895
average bypass improvement is  0.056681480012018726
max bypass improvement is  0.150960815196449
